# Data Mining / Prospecção de Dados

## Sara C. Madeira, 2024/2025

# Project 1 - Pattern Mining

## Logistics 
**_Read Carefully_**

**Students should work in teams of 3 people**. 

Groups with less than 3 people might be allowed (with valid justification), but will not have better grades for this reason. 

The quality of the project will dictate its grade, not the number of people working.

**The project's solution should be uploaded in Moodle before the end of `May, 4th (23:59)`.** 

Students should **upload a `.zip` file** containing a folder with all the files necessary for project evaluation. 
Groups should be registered in [Moodle](https://moodle.ciencias.ulisboa.pt/mod/groupselect/view.php?id=139096) and the `zip` file should be identified as `PDnn.zip` where `nn` is the number of your group.

**It is mandatory to produce a Jupyter notebook containing code and text/images/tables/etc describing the solution and the results. Projects not delivered in this format will not be graded. You can use `PD_202425_P1.ipynb` as template. In your `.zip` folder you should also include an HTML version of your notebook with all the outputs.**

**Decisions should be justified and results should be critically discussed.** 

Remember that **your notebook should be as clear and organized as possible**, that is, **only the relevant code and experiments should be presented, not everything you tried and did not work, or is not relevant** (that can be discussed in the text, if relevant)! Tables and figures can be used together with text to summarize results and conclusions, improving understanding, readability and concision. **More does not mean better! The target is quality not quantity!**

_**Project solutions containing only code and outputs without discussions will achieve a maximum grade of 10 out of 20.**_

## Dataset and Tools

The dataset to be analysed is **`Foodmart_2025_DM.csv`**, which is a modified and integrated version of the **Foodmart database**, used in several [Kaggle](https://www.kaggle.com) Pattern Mining competitions, with the goal of finding **actionable patterns** by analysing data from the `FOODmart Ltd` company, a leading supermarket chain. 

`FOODmart Ltd` has different types of stores: Deluxe Supermarkets, Gourmet Supermarkets, Mid-Size Grocerys, Small Grocerys and 
Supermarkets. Y

Your **goals** are to find: 
1. **global patterns** (common to all stores) and
2. **local/specific patterns** (related to the type of store).

**`Foodmart_2025_DM.csv`** stores **69549 transactions** from **24 stores**, where **103 different products** can be bought. 

Each transaction (row) has a `STORE_ID` (integer from 1 to 24), and a list of produts (items), together with the quantities bought. 

In the transation highlighted below, a given customer bought 1 unit of soup, 2 of cheese and 1 of wine at store 2.

<img src="Foodmart_2025_DM_Example.png" alt="Foodmart_2025_DM_Example" style="width: 1000px;"/>

In this context, the project has **2 main tasks**:
1. Mining Frequent Itemsets and Association Rules: Ignoring Product Quantities and Stores **(global patterns)**
2. Mining Frequent Itemsets and Association Rules: Looking for Differences between Stores **(local/specific patterns)**

# **While doing PATTERN and ASSOCIATION MINING keep in mind the following basic/key questions and BE CREATIVE!**

### 1. What are the most popular products?
* I think we could make a semantics to label items, for instance, foods as healthy or unhealthy and such stuff like that to kinda have a more general notion on what people are buying!!!
    
### 3. Which products are bought together?
### 4. What are the frequent patterns?
### 5. Can we find associations highlighting that when people buy a product/set of products also buy other product(s)?
### 6. Are these associations strong? Can we trust them? Are they misleading?
### 7. Can we analyse these patterns and evaluate these associations to find, not only frequent and strong associations, but also interest patterns and associations?

**In this project you should use [Python 3](https://www.python.org), [Jupyter Notebook](http://jupyter.org) and [`MLxtend`](http://rasbt.github.io/mlxtend/).**

When using `MLxtend`, frequent patterns can either be discovered using `Apriori` and `FP-Growth`. **Choose the pattern mining algorithm to be used.** 

FP-growth is more efficient with larger datasets than apriori; however it can be memory-intensive, especially for datasets with many frequent items. 


## Team Identification

**GROUP 09**

Students:

* Daniel João - *56455*
* Daniel Ribeiro - *64476*
* Miguel Alves - *52724* 

## 1. Mining Frequent Itemsets and Association Rules: Ignoring Product Quantities and Stores

In this first task you should load and preprocessed the dataset **`Foodmart_2025_DM.csv`** in order to compute frequent itemsets and generate association rules considering all the transactions, regardeless of the store, and ignoring product quantities.

We might need to install an older version of *mlxtend* cuz of the issue we saw on the TP classes:

In [1]:
#!pip install mlxtend==0.19

I recommend that all use a conda environment to keep these packages separate:
(in terminal/ bash)
* 1. Create a conda env: **"conda create -n env_name python=3.9"**
* 2. Activate your conda environment: **"conda activate env_name"**
* 3. Install the correct package version: **"conda install -c conda-forge mlxtend=0.19"**
* 4. Install other packages like pandas and such ...
* 5. Install kernel package so you can use this env has a kernel on Jupyter Notebook: **"conda install -c conda-forge ipykernel"**

### 1.0. Used Packages Importing

In [2]:
import csv
import re
import io
import pandas as pd
import seaborn as sns
import time

# mining patterns
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import fpgrowth

**Set pandas to display all columns of a df:**

In [3]:
pd.set_option('display.max_columns', None)

### 1.1. Load and Preprocess Dataset

**Product quantities and stores should not be considered.**

- Removing STORE_ID info; each row is a transaction.
- Values transformation, from Integer to Boolean (One-Hot Encoding):
    - **True** if value > 1,
    - **False** if value = 0.
- each row is a transaction.

#### 1.1.1. Fixing Malformed Lines 

<ins>Fifty-four lines in the CSV are malformed</ins>, having **missing commas** between product types like the following:

In [4]:
filename = "Foodmart_2025_DM.csv"

In [5]:
pattern = re.compile(r'(?<=[0-9])(?=[A-Za-z])')

# Read the file lines
with open(filename, "r", encoding="utf-8") as f:
    lines = f.readlines()

# Find lines that match the pattern
bad_lines = [line.strip() for line in lines if pattern.search(line)]

# Print the first 10 problematic lines
print("Lines with missing commas (showing first 10):")
for line in bad_lines[:10]:
    print(line)

print(len(bad_lines))

Lines with missing commas (showing first 10):
STORE_ID=2,Soup=1Fresh Vegetables=2,Nasal Sprays=2
STORE_ID=2,Soup=2Milk=1,Deli Meats=1,TV Dinner=2,Pancakes=2
STORE_ID=2,Soup=3Fresh Vegetables=3,Cottage Cheese=1,Juice=2,Frozen Vegetables=2
STORE_ID=2,Soup=1Fresh Vegetables=2,Dips=2,Fresh Fruit=1,Dried Fruit=2,Paper Wipes=1,Anchovies=1
STORE_ID=2,Soup=3Fresh Fruit=2,Lightbulbs=3,Wine=2
STORE_ID=2,Soup=2Eggs=1,Personal Hygiene=2,Fresh Fruit=1,Juice=1
STORE_ID=2,Pasta=2Waffles=2,Nuts=2
STORE_ID=2,Soup=2Eggs=3
STORE_ID=2,Soup=2Cereal=1,Juice=1,Ice Cream=1
STORE_ID=2,Soup=1Fresh Vegetables=2,Frozen Chicken=2,Sour Cream=1,Clams=2
54


##### Fixing that (no number should be immediately before a letter, since the malformed lines only have letters after the number) and opening content in the panda's data frame:

In [6]:
def process_transaction_csv(filename):
    """
    Reads the CSV file containing transactions, fixes missing commas between key-value pairs,
    processes the transactions, and returns a list of transactions.
    Requires:
        - The CSV file at `filename` is accessible and readable.
        - Each line in the file is formatted with key-value pairs in the format "Key=Value".
        - Some rows may be missing commas between consecutive key-value pairs
          (e.g., "Pasta=2Waffles=2" should be "Pasta=2,Waffles=2").
        - The value for each key is numeric, and only keys with a numeric value above 0 should be considered.
        - Keys that start with a specific prefix (e.g., "STORE_ID") are considered auxiliary and will be ignored.
    Ensures: The function returns a list of transactions, where each transaction is a list of keys (products) 
    that had a numeric value greater than 0.
    Returns: list: A list of transactions, each transaction is a list of product keys.
    """

    with open(filename, "r", encoding="utf-8") as f:
        data = f.read()
    data_fixed = re.sub(r'(?<=[0-9])(?=[A-Za-z])', ',', data)
    
    # Create a file-like object from the fixed data.
    csvfile = io.StringIO(data_fixed)
    reader = csv.reader(csvfile, delimiter=',')
    
    dataset = []
    for row in reader:
        transaction = []
        for kv in row:
            if '=' in kv:
                key, value = kv.split('=', 1)
                key = key.strip()
                value = value.strip()
                
                if key.startswith("STORE_ID"):
                    continue
                try:
                    numeric_value = float(value)
                except ValueError:
                    continue
                if numeric_value > 0:
                    transaction.append(key)
        dataset.append(transaction)
    
    return dataset

dataset = process_transaction_csv(filename)
te = TransactionEncoder()
te_ary = te.fit_transform(dataset)
binary_foodmart = pd.DataFrame(te_ary, columns=te.columns_)

binary_foodmart.head(10)

,Acetominifen,Anchovies,Aspirin,Auto Magazines,Bagels,Batteries,Beer,Bologna,Candles,Canned Fruit,Canned Vegetables,Cereal,Cheese,Chips,Chocolate,Chocolate Candy,Clams,Cleaners,Coffee,Cold Remedies,Computer Magazines,Conditioner,Cookies,Cooking Oil,Cottage Cheese,Crackers,Deli Meats,Deli Salads,Deodorizers,Dips,Donuts,Dried Fruit,Dried Meat,Eggs,Fashion Magazines,Flavored Drinks,French Fries,Fresh Chicken,Fresh Fish,Fresh Fruit,Fresh Vegetables,Frozen Chicken,Frozen Vegetables,Gum,Hamburger,Hard Candy,Home Magazines,Hot Dogs,Ibuprofen,Ice Cream,Jam,Jelly,Juice,Lightbulbs,Maps,Milk,Mouthwash,Muffins,Nasal Sprays,Nuts,Oysters,Pancake Mix,Pancakes,Paper Dishes,Paper Wipes,Pasta,Peanut Butter,Personal Hygiene,Pizza,Plastic Utensils,Popcorn,Popsicles,Pot Cleaners,Pot Scrubbers,Pots and Pans,Preserves,Pretzels,Rice,Sardines,Sauces,Screwdrivers,Shampoo,Shellfish,Shrimp,Sliced Bread,Soda,Soup,Sour Cream,Spices,Sponges,Sports Magazines,Sugar,Sunglasses,TV Dinner,Tofu,Toilet Brushes,Tools,Toothbrushes,Tuna,Waffles,Wine,Yogurt
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,Fal

#### 1.1.2. Foodmart Basic Stats

##### Lines and Columns

- Transcation_number = 69549
- Number of Total Product Types/ Columns = 102

In [7]:
binary_foodmart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69549 entries, 0 to 69548
Columns: 102 entries, Acetominifen to Yogurt
dtypes: bool(102)
memory usage: 6.8 MB


##### Are there empty transactions?

In [8]:
binary_foodmart[binary_foodmart.sum(axis=1) == 0]

,Acetominifen,Anchovies,Aspirin,Auto Magazines,Bagels,Batteries,Beer,Bologna,Candles,Canned Fruit,Canned Vegetables,Cereal,Cheese,Chips,Chocolate,Chocolate Candy,Clams,Cleaners,Coffee,Cold Remedies,Computer Magazines,Conditioner,Cookies,Cooking Oil,Cottage Cheese,Crackers,Deli Meats,Deli Salads,Deodorizers,Dips,Donuts,Dried Fruit,Dried Meat,Eggs,Fashion Magazines,Flavored Drinks,French Fries,Fresh Chicken,Fresh Fish,Fresh Fruit,Fresh Vegetables,Frozen Chicken,Frozen Vegetables,Gum,Hamburger,Hard Candy,Home Magazines,Hot Dogs,Ibuprofen,Ice Cream,Jam,Jelly,Juice,Lightbulbs,Maps,Milk,Mouthwash,Muffins,Nasal Sprays,Nuts,Oysters,Pancake Mix,Pancakes,Paper Dishes,Paper Wipes,Pasta,Peanut Butter,Personal Hygiene,Pizza,Plastic Utensils,Popcorn,Popsicles,Pot Cleaners,Pot Scrubbers,Pots and Pans,Preserves,Pretzels,Rice,Sardines,Sauces,Screwdrivers,Shampoo,Shellfish,Shrimp,Sliced Bread,Soda,Soup,Sour Cream,Spices,Sponges,Sports Magazines,Sugar,Sunglasses,TV Dinner,Tofu,Toilet Brushes,Tools,Toothbrushes,Tuna,Waffles,Wine,Yogurt


In [9]:
empty_count = 0
for check in dataset:
    if check == []: 
        empty_countcount += 1
if empty_count == 0:
    print('No empty transactions.')

No empty transactions.


##### Mean, maximum, minimum and median products per transaction

In [10]:
# each True is treated as 1
product_counts = binary_foodmart.sum(axis=1)

average_products = product_counts.mean()
max_products = product_counts.max()
min_products = product_counts.min()
median_products = product_counts.median()

print("Mean products per transaction:", average_products)
print("Maximum products in a transaction:", max_products)
print("Minimum products in a transaction:", min_products)
print("Median products per transaction:", median_products)

Mean products per transaction: 4.158435060173403
Maximum products in a transaction: 13
Minimum products in a transaction: 1
Median products per transaction: 4.0


### 1.2. Compute Frequent Itemsets

* Compute frequent itemsets considering a minimum support S_min. 
* Present frequent itemsets organized by length (number of items). 
* List frequent 1-itemsets, 2-itemsets, 3-itemsets, etc with support of at least S < S_min.
* Change the minimum support values and discuss the results.

In [11]:
# higher support items will appear first; descendent order
def compute_frequent_itemset_subsets(df, min_support, algorithm='fp_growth'):
    """
    Compute frequent itemsets from a one-hot encoded DataFrame using a specified algorithm and a minimum support threshold,
    and return the frequent itemsets organizsd by their length as separate DataFrames.
    Parameters:
    df : pandas.DataFrame
        A one-hot encoded DataFrame where each row represents a transaction and each column
        represents an item as a boolean value (True/False).
    min_support : float
        A value between 0 and 1 specifies the minimum fraction of transactions in which an itemset 
        must appear to be considered frequent.
    algorithm: str, optional (default='apriori')
        The algorithm to be used for frequent itemset mining. Supported options are 'apriori' and 'fp_growth'.
    Returns:
        A dictionary mapping each itemset length (e.g., 1, 2, 3, …) to a Ddata fame containing the frequent itemsets
        of that length, with their corresponding support values,
        ordered in descending order so that itemsets with higher support appear first.
    """
    if algorithm == 'apriori':
        frequent_itemsets = apriori(df, min_support=min_support, use_colnames=True)
    elif algorithm == 'fp_growth':
        frequent_itemsets = fpgrowth(df, min_support=min_support, use_colnames=True)
    else:
        raise ValueError("Unsupported algorithm. Please use 'apriori' or 'fp_growth'.")
    
    frequent_itemsets['itemset_length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
    
    # Organize and sort the frequent itemsets by their length.
    subsets = {}
    for length in sorted(frequent_itemsets['itemset_length'].unique()):
        subset_df = frequent_itemsets[frequent_itemsets['itemset_length'] == length]
        # Order the itemset subset by support in descending order.
        subset_df = subset_df.sort_values(by='support', ascending=False)
        subsets[length] = subset_df
        
    return subsets


**Visualise ≠ length itemsets with a key that equals their itemset partition length.**

#### 1.2.1. Minimum Support Threshold

<ins>Support is the **proportion of times an item appears**,</ins> meaning if an item appears in 5 out of 10 transactions, its support is 0.5 or 50%.

##### Minimum Support Threshold == 5%

In [12]:
S_min_high = 0.05
frequent_subsets_high = compute_frequent_itemset_subsets(binary_foodmart, S_min_high)
print("Frequent itemsets can be partioned in",len(frequent_subsets_high), "itemsets of differing lengths.")
print(frequent_subsets_high)

Frequent itemsets can be partioned in 2 itemsets of differing lengths.
{np.int64(1):      support             itemsets  itemset_length
1   0.284461   (Fresh Vegetables)               1
12  0.175286        (Fresh Fruit)               1
0   0.120059               (Soup)               1
3   0.117845             (Cheese)               1
14  0.117227        (Dried Fruit)               1
4   0.105422            (Cookies)               1
29  0.080677               (Wine)               1
15  0.078966        (Paper Wipes)               1
20  0.078549  (Canned Vegetables)               1
22  0.068340  (Frozen Vegetables)               1
28  0.067176               (Nuts)               1
16  0.066744    (Chocolate Candy)               1
2   0.066342               (Milk)               1
5   0.066040          (Preserves)               1
18  0.064746              (Chips)               1
6   0.064070               (Eggs)               1
13  0.056320       (Sliced Bread)               1
24  0.056277   

In [13]:
print("Frequent itemsets of 1 item")
frequent_subsets_high[1]

Frequent itemsets of 1 item


,support,itemsets,itemset_length
1,0.284461,(Fresh Vegetables),1
12,0.175286,(Fresh Fruit),1
0,0.120059,(Soup),1
3,0.117845,(Cheese),1
14,0.117227,(Dried Fruit),1
4,0.105422,(Cookies),1
29,0.080677,(Wine),1
15,0.078966,(Paper Wipes),1
20,0.078549,(Canned Vegetables),1
22,0.068340,(Frozen Vegetables),1


In [14]:
print("Frequent itemsets of 2 items")
frequent_subsets_high[2]

Frequent itemsets of 2 items


,support,itemsets,itemset_length
31,0.050971,"(Fresh Vegetables, Fresh Fruit)",2


##### **Mininum Support Threshhold = 1%**

In [15]:
S_min_high = 0.01
frequent_subsets_high = compute_frequent_itemset_subsets(binary_foodmart, S_min_high)
print("Frequent itemsets can be partioned in",len(frequent_subsets_high), "itemsets of differing lengths.")

Frequent itemsets can be partioned in 2 itemsets of differing lengths.


In [16]:
print("Frequent itemsets of 1 item")
frequent_subsets_high[1]

Frequent itemsets of 1 item


,support,itemsets,itemset_length
2,0.284461,(Fresh Vegetables),1
29,0.175286,(Fresh Fruit),1
0,0.120059,(Soup),1
5,0.117845,(Cheese),1
32,0.117227,(Dried Fruit),1
...,...,...,...
100,0.013343,(Sardines),1
34,0.013271,(Sauces),1
83,0.013156,(Fashion Magazines),1
95,0.012998,(Pancakes),1


In [17]:
print("Frequent itemsets of 2 items")
frequent_subsets_high[2]

Frequent itemsets of 2 items


,support,itemsets,itemset_length
134,0.050971,"(Fresh Vegetables, Fresh Fruit)",2
102,0.035701,"(Fresh Vegetables, Soup)",2
138,0.035284,"(Fresh Vegetables, Dried Fruit)",2
110,0.031201,"(Fresh Vegetables, Cheese)",2
112,0.027765,"(Fresh Vegetables, Cookies)",2
...,...,...,...
121,0.010281,"(Fresh Vegetables, Cleaners)",2
157,0.010194,"(Frozen Vegetables, Fresh Fruit)",2
155,0.010165,"(Fresh Vegetables, Popcorn)",2
175,0.010137,"(Wine, Dried Fruit)",2


##### Minimum Support Threshold = 0.1%


Much faster by using FP-Growth

As expected, when the minimum support threshold is as low as 0.1%, the FP-Growth algorithm demonstrates significantly faster performance compared to Apriori because it leverages an efficient data structure known as the FP tree. 

Unlike the Apriori algorithm that generates candidate itemsets, FP-growth compacts the representation of the transactional dataset, minimising the need for exhaustive candidate generation, and allowing the algorithm to bypass the combinatorial explosion. 

Therefore, fp-growth quickly identifies frequent patterns even amidst a large number of potential itemsets, leading to improvements in execution time under such low support conditions. The improvements are plotted in the following graph:

In [ ]:
support_thresholds = [0.05, 0.03, 0.01, 0.005, 0.001]
results = []

for support in support_thresholds:
    start_time = time.time()
    _ = compute_frequent_itemset_subsets(binary_foodmart, min_support=support, algorithm='apriori')
    apriori_time = time.time() - start_time
    results.append({'Support': support * 100, 'Time': apriori_time, 'Algorithm': 'Apriori'})
    
    start_time = time.time()
    _ = compute_frequent_itemset_subsets(binary_foodmart, min_support=support, algorithm='fp_growth')
    fpgrowth_time = time.time() - start_time
    results.append({'Support': support * 100, 'Time': fpgrowth_time, 'Algorithm': 'FP-Growth'})


In [ ]:
df_times = pd.DataFrame(results)

sns.set_style("darkgrid", {"axes.facecolor": ".9"})
plt.figure(figsize=(8, 6))

ax = sns.lineplot(data=df_times, x='Support', y='Time', hue='Algorithm', marker='o', palette="deep")
ax.set_title('Execution Time: Apriori vs FP-Growth')
ax.set_xlabel('Minimum Support Threshold (%)')
ax.set_ylabel('Execution Time (seconds)')
plt.gca().invert_xaxis() 
plt.show()

In [42]:
S_min_high = 0.001
frequent_subsets_high = compute_frequent_itemset_subsets(binary_foodmart, S_min_high, "fp_growth")
print("Frequent itemsets can be partioned in",len(frequent_subsets_high), "itemsets of differing lengths.")

Frequent itemsets can be partioned in 3 itemsets of differing lengths.


In [43]:
print("Frequent itemsets of 1 item")
frequent_subsets_high[1]

Frequent itemsets of 1 item


,support,itemsets,itemset_length
2,0.284461,(Fresh Vegetables),1
29,0.175286,(Fresh Fruit),1
0,0.120059,(Soup),1
5,0.117845,(Cheese),1
32,0.117227,(Dried Fruit),1
...,...,...,...
100,0.013343,(Sardines),1
34,0.013271,(Sauces),1
83,0.013156,(Fashion Magazines),1
95,0.012998,(Pancakes),1


In [44]:
print("Frequent itemsets of 2 items")
frequent_subsets_high[2]

Frequent itemsets of 2 items


,support,itemsets,itemset_length
1121,0.050971,"(Fresh Vegetables, Fresh Fruit)",2
102,0.035701,"(Fresh Vegetables, Soup)",2
1193,0.035284,"(Fresh Vegetables, Dried Fruit)",2
222,0.031201,"(Fresh Vegetables, Cheese)",2
327,0.027765,"(Fresh Vegetables, Cookies)",2
...,...,...,...
1942,0.001006,"(Tools, Flavored Drinks)",2
2915,0.001006,"(Cereal, Conditioner)",2
592,0.001006,"(Preserves, Tofu)",2
594,0.001006,"(Tofu, Sour Cream)",2


In [45]:
print("Frequent itemsets of 3 items")
frequent_subsets_high[3]

Frequent itemsets of 3 items


,support,itemsets,itemset_length
104,0.007089,"(Fresh Vegetables, Soup, Fresh Fruit)",3
332,0.005550,"(Fresh Vegetables, Fresh Fruit, Cookies)",3
225,0.005334,"(Fresh Vegetables, Cheese, Fresh Fruit)",3
1200,0.004975,"(Fresh Vegetables, Fresh Fruit, Dried Fruit)",3
1213,0.004702,"(Fresh Vegetables, Paper Wipes, Fresh Fruit)",3
...,...,...,...
863,0.001006,"(Fresh Vegetables, French Fries, Soup)",3
2305,0.001006,"(Chips, Fresh Vegetables, Deli Salads)",3
313,0.001006,"(Hard Candy, Fresh Vegetables, Soup)",3
2658,0.001006,"(Fresh Vegetables, Fashion Magazines, Fresh Fr...",3


#### 1.2.2. Discussing Results

Initially, setting the minimum support threshold to 5% resulted in a relatively limited set of frequent itemsets. In this configuration, only strong individual items emerged as frequent, and a single pair—comprising fresh fruit and fresh vegetables—was identified as a frequent 2-itemset. This outcome underscores that a higher support level filters out less common or weaker associations, thereby ensuring that only the most dominant patterns, which appear in a substantial fraction of transactions, are retained. This characteristic is particularly beneficial when the analysis seeks to focus exclusively on the most robust and reliable associations.

As the threshold was lowered to 1%, a notable increase in the number of detected frequent itemsets was observed. The set of frequent 1-itemsets expanded significantly to include a broader range of items, encompassing both commonly purchased items and those with lower individual occurrence rates. Moreover, the discovery of 2-itemsets increased, revealing more nuanced co-occurrence patterns between items that may not have been captured under the higher threshold. This moderate threshold setting offers a compromise that balances comprehensiveness and reliability. It permits the identification of additional associations that, while less frequent than those found at a 5% threshold, may still hold significant business value depending on the context and objectives of the analysis.

When the support threshold was further reduced to 0.1%, the results demonstrated an even more dramatic increase in the number of frequent itemsets, including the emergence of 3-itemsets. Under these conditions, the algorithm captured a vast array of item combinations, revealing intricate and potentially insightful interactions among products. However, the extension of the itemset scope to include lower support levels also introduces a challenge: the potential inclusion of noise or statistically insignificant patterns. The resulting explosion in the number of itemsets can increase computational complexity and demand additional filtering steps to discern which of the many associations are truly meaningful for the project.

In conclusion, the findings illustrate that the choice of minimum support threshold is a critical factor in itemset mining. A higher threshold tends to emphasize the most dominant patterns and minimizes the risk of incorporating spurious relationships, which is advantageous for analyses focused on the most impactful associations. Conversely, employing a lower threshold broadens the scope of discovery, capturing a richer set of relationships that may reveal niche or emergent patterns, though at the expense of increased computational complexity and the necessity for subsequent refinement. The results of these experiments thus provide essential guidance in selecting an appropriate threshold that aligns with the project’s analytical goals, whether the focus is on identifying core, high-confidence item associations or exploring a comprehensive landscape of potential interactions for further investigation.

### 1.3. Generate Association Rules from Frequent Itemsets

Using a minimum support S_min fundamented by the previous results. 
* Generate association rules with a choosed value (C) for minimum confidence. 
* Generate association rules with a choosed value (L) for minimum lift. 
* Generate association rules with both confidence >= C and lift >= L.
* Change C and L when it makes sense and discuss the results.
* Use other metrics besides confidence and lift.
* Evaluate how good the rules are given the metrics and how interesting they are from your point of view.

#### 1.3.1 Association Rules for Confidence

**Since the support minimum threshold level of 5% didn't display but one pair of items, the value chosen for the association rules for confidence was lowered to 1%, a percentage that has relatively plenty of item pairs to display — which will form the association rules —, and also a relatively fast computation using FP-Growth. Now, let's form the confidence association rules.**

In [62]:
from mlxtend.frequent_patterns import association_rules

supp = 0.01

frequent_subsets = fpgrowth(binary_foodmart, min_support=supp)

#Now the association rules
conf = 0.1
rules_01 = association_rules(frequent_subsets, metric="confidence", min_threshold=conf)

# Print the generated association rules for confidence
rules_01
rules_01.sort_values(by='confidence', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
72,(81),(40),0.041007,0.284461,0.012667,0.308906,1.085933,0.001002,1.035371
39,(30),(40),0.040403,0.284461,0.012466,0.308541,1.084650,0.000973,1.034824
35,(67),(40),0.054623,0.284461,0.016506,0.302185,1.062305,0.000968,1.025399
46,(31),(40),0.117227,0.284461,0.035284,0.300993,1.058118,0.001938,1.023651
36,(47),(40),0.040432,0.284461,0.012107,0.299431,1.052625,0.000605,1.021368
...,...,...,...,...,...,...,...,...,...
19,(86),(22),0.120059,0.105422,0.012639,0.105269,0.998552,-0.000018,0.999829
49,(39),(31),0.175286,0.117227,0.018318,0.104503,0.891463,-0.002230,0.985792
48,(31),(86),0.117227,0.120059,0.012193,0.104011,0.866329,-0.001881,0.982089
47,(86),(31),0.120059,0.117227,0.012193,0.101557,0.866329,-0.001881,0.982559


**We have 89 association rules with a minimum confidence of 0.1 and going up to >0.3. So, let's create association rules with a minimum confidence value up to 0.3.**
**First we will create the function to retrieve the antecedents and consequent item names with the respective confidence level.**

In [63]:
# Function to get item names from indices 

def get_formatted_rules(rules_df, binary_data):
    """
    Formats an existing DataFrame of association rules with item names
    and selects specific columns.

    Args:
        rules_df (pd.DataFrame): DataFrame of association rules
                                   generated by association_rules.
        binary_data (pd.DataFrame): Original binary encoded DataFrame
                                     where columns are item names.

    Returns:
        pd.DataFrame: Formatted DataFrame of association rules with item names,
                      antecedent support, consequent support, and confidence.
    """
    item_names = binary_data.columns

    def get_item_names(item_indices, data_columns):
        items = [data_columns[i] for i in item_indices]
        return frozenset(items)

    rules_df['antecedents_names'] = rules_df['antecedents'].apply(lambda x: get_item_names(x, item_names))
    rules_df['consequents_names'] = rules_df['consequents'].apply(lambda x: get_item_names(x, item_names))

    formatted_rules = rules_df[['antecedents_names', 'consequents_names', 'antecedent support', 'consequent support', 'confidence']]

    return formatted_rules

In [67]:
#Retriving the first association rules with confidence minimum=0.1
get_formatted_rules(rules_01, binary_foodmart)

,antecedents_names,consequents_names,antecedent support,consequent support,confidence
0,(Fresh Vegetables),(Soup),0.284461,0.120059,0.125505
1,(Soup),(Fresh Vegetables),0.120059,0.284461,0.297365
2,(Soup),(Fresh Fruit),0.120059,0.175286,0.173293
3,(Fresh Fruit),(Soup),0.175286,0.120059,0.118694
4,(Pasta),(Fresh Vegetables),0.049217,0.284461,0.270815
...,...,...,...,...,...
84,(Wine),(Fresh Vegetables),0.080677,0.284461,0.254500
85,(Wine),(Fresh Fruit),0.080677,0.175286,0.185707
86,(Wine),(Dried Fruit),0.080677,0.117227,0.125646
87,(Pizza),(Fresh Vegetables),0.054048,0.284461,0.283054


**Now we'll create a for loop for the different confidence minimum thresholds up to 0.3.**

In [65]:
# To generate the association rules let's use a for loop with different confidence levels (0.1, 0.15, 0.2, 0.3)

# Define the list of lift thresholds you want to use
confidence_thresholds = [0.1, 0.15, 0.2, 0.3]

# Dictionary to store formatted rules for each confidence threshold
formatted_rules_by_conf = {}

# Loop over each confidence value, get the rules and print them
for conf_value in confidence_thresholds:
    # Get association rules with the current confidence threshold
    rules = association_rules(frequent_subsets, metric="confidence", min_threshold=conf_value)
    
    # Format the rules
    formatted_rules = get_formatted_rules(rules, binary_foodmart)
    
    # Save the formatted rules into the dictionary
    formatted_rules_by_conf[conf_value] = formatted_rules
    
    # Display the results
    print(f"Formatted rules for lift threshold = {conf_value}:")
    print(formatted_rules)
    print("\n" + "-"*50 + "\n")

Formatted rules for lift threshold = 0.1:
     antecedents_names   consequents_names  antecedent support  \
0   (Fresh Vegetables)              (Soup)            0.284461   
1               (Soup)  (Fresh Vegetables)            0.120059   
2               (Soup)       (Fresh Fruit)            0.120059   
3        (Fresh Fruit)              (Soup)            0.175286   
4              (Pasta)  (Fresh Vegetables)            0.049217   
..                 ...                 ...                 ...   
84              (Wine)  (Fresh Vegetables)            0.080677   
85              (Wine)       (Fresh Fruit)            0.080677   
86              (Wine)       (Dried Fruit)            0.080677   
87             (Pizza)  (Fresh Vegetables)            0.054048   
88             (Pizza)       (Fresh Fruit)            0.054048   

    consequent support  confidence  
0             0.120059    0.125505  
1             0.284461    0.297365  
2             0.175286    0.173293  
3             0.1

**It's sad that we lose the beautiful interface by using the ***print*** function in our loop...**

#### **Discussion of these results**
All rules from minimum_confidence=0.25 predict the purchase of "Fresh Vegetables", the most frequent item (28.45%). Confidence values range up to to 30.89%, indicating weak positive associations. Some rules (e.g., Donuts or Shampoo → Fresh Vegetables) show slightly above average confidence but aren't strong predictors. The frequent appearance of this item as a consequent likely reflects its popularity rather than meaningful connections, so, to confirm there is indeed strong relationships, further evaluatom will continue with lift association rules.

#### *1.3.2 Association Rules for Lift (without confidence)*

**Let's start with a lift minimum of 1 and raise it from there, since levels > 1 imply that the antecedent influences the consequent beyond random chance.**

In [66]:
lift = 1
rules_1 = association_rules(frequent_subsets, metric="lift", min_threshold=lift)
rules_1

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(40),(86),0.284461,0.120059,0.035701,0.125505,1.045363,0.001549,1.006228
1,(86),(40),0.120059,0.284461,0.035701,0.297365,1.045363,0.001549,1.018365
2,(39),(55),0.175286,0.066342,0.012337,0.070380,1.060868,0.000708,1.004344
3,(55),(39),0.066342,0.175286,0.012337,0.185956,1.060868,0.000708,1.013107
4,(12),(86),0.117845,0.120059,0.015227,0.129209,1.076213,0.001078,1.010508
...,...,...,...,...,...,...,...,...,...
65,(39),(100),0.175286,0.080677,0.014982,0.085473,1.059446,0.000841,1.005244
66,(100),(31),0.080677,0.117227,0.010137,0.125646,1.071821,0.000679,1.009629
67,(31),(100),0.117227,0.080677,0.010137,0.086471,1.071821,0.000679,1.006343
68,(68),(39),0.054048,0.175286,0.010654,0.197127,1.124598,0.001180,1.027203


**We'll use the same formatting function using before in the confidence association rules adapted for the lift.**

In [38]:
# Function to get item names from indices 

def get_formatted_rules(rules_df, binary_data):
    """
    Formats an existing DataFrame of association rules with item names
    and selects specific columns.

    Args:
        rules_df (pd.DataFrame): DataFrame of association rules
                                   generated by association_rules.
        binary_data (pd.DataFrame): Original binary encoded DataFrame
                                     where columns are item names.

    Returns:
        pd.DataFrame: Formatted DataFrame of association rules with item names,
                      antecedent support, consequent support, and confidence.
    """
    item_names = binary_data.columns

    def get_item_names(item_indices, data_columns):
        items = [data_columns[i] for i in item_indices]
        return frozenset(items)

    rules_df['antecedents_names'] = rules_df['antecedents'].apply(lambda x: get_item_names(x, item_names))
    rules_df['consequents_names'] = rules_df['consequents'].apply(lambda x: get_item_names(x, item_names))

    formatted_rules = rules_df[['antecedents_names', 'consequents_names', 'antecedent support', 'consequent support', 'lift']]

    return formatted_rules

In [68]:
#Let's get our see the items for our first lift associatin rules
format_rules = get_formatted_rules(rules_1, binary_foodmart)
format_rules

,antecedents_names,consequents_names,antecedent support,consequent support,confidence
0,(Fresh Vegetables),(Soup),0.284461,0.120059,0.125505
1,(Soup),(Fresh Vegetables),0.120059,0.284461,0.297365
2,(Fresh Fruit),(Milk),0.175286,0.066342,0.070380
3,(Milk),(Fresh Fruit),0.066342,0.175286,0.185956
4,(Cheese),(Soup),0.117845,0.120059,0.129209
...,...,...,...,...,...
65,(Fresh Fruit),(Wine),0.175286,0.080677,0.085473
66,(Wine),(Dried Fruit),0.080677,0.117227,0.125646
67,(Dried Fruit),(Wine),0.117227,0.080677,0.086471
68,(Pizza),(Fresh Fruit),0.054048,0.175286,0.197127


In [50]:
#Let's see the first 20 highest lift values and adjust our settings accordingly
ascend = format_rules.sort_values(by='lift', ascending=False)
ascend.head(20)

,antecedents_names,consequents_names,antecedent support,consequent support,lift
63,(Soup),(Wine),0.120059,0.080677,1.163805
62,(Wine),(Soup),0.080677,0.120059,1.163805
43,(Fresh Fruit),(Juice),0.175286,0.053689,1.144350
42,(Juice),(Fresh Fruit),0.053689,0.175286,1.144350
52,(Batteries),(Fresh Fruit),0.053962,0.175286,1.141597
53,(Fresh Fruit),(Batteries),0.175286,0.053962,1.141597
69,(Fresh Fruit),(Pizza),0.175286,0.054048,1.124598
68,(Pizza),(Fresh Fruit),0.054048,0.175286,1.124598
26,(Sliced Bread),(Fresh Fruit),0.056320,0.175286,1.108365
27,(Fresh Fruit),(Sliced Bread),0.175286,0.056320,1.108365


**Now let's code the for loop function that iterates through the different lift minimum thresholds, which we will set from 1.03 (to raise it a bit from 1) up to a minimum of 1.1 (where we have the 10 highest lift association rules.)**

In [69]:
#So let's raise lift in the association rules up to 1.10 where we have 10 pairs of items ot analyse. Let's make a loop for that.

# Define the list of lift thresholds you want to use
lift_thresholds = [1.03, 1.05, 1.1]

# Dictionary to store formatted rules for each lift threshold
formatted_rules_by_lift = {}

# Loop over each lift value, get the rules and print them
for lift_value in lift_thresholds:
    # Get association rules with the current lift threshold
    rules = association_rules(frequent_subsets, metric="lift", min_threshold=lift_value)
    
    # Format the rules
    formatted_rules = get_formatted_rules(rules, binary_foodmart)
    
    # Save the formatted rules into the dictionary
    formatted_rules_by_lift[lift_value] = formatted_rules
    
    # Display the results
    print(f"Formatted rules for lift threshold = {lift_value}:")
    print(formatted_rules)
    print("\n" + "-"*50 + "\n")

Formatted rules for lift threshold = 1.03:
     antecedents_names   consequents_names  antecedent support  \
0   (Fresh Vegetables)              (Soup)            0.284461   
1               (Soup)  (Fresh Vegetables)            0.120059   
2        (Fresh Fruit)              (Milk)            0.175286   
3               (Milk)       (Fresh Fruit)            0.066342   
4             (Cheese)              (Soup)            0.117845   
5               (Soup)            (Cheese)            0.120059   
6            (Cookies)       (Dried Fruit)            0.105422   
7        (Dried Fruit)           (Cookies)            0.117227   
8   (Fresh Vegetables)        (Deli Meats)            0.284461   
9         (Deli Meats)  (Fresh Vegetables)            0.053617   
10  (Fresh Vegetables)  (Personal Hygiene)            0.284461   
11  (Personal Hygiene)  (Fresh Vegetables)            0.054623   
12  (Fresh Vegetables)          (Hot Dogs)            0.284461   
13          (Hot Dogs)  (Fresh Ve

**We narrowed 48 pairs of antecedens and consequents down to 10 pairs of potential stronger relationships, from raisng the lift=1.03 to lfit=1.1. Now let's check the relative robust association rules by utilizing both confidence and lift.**

#### 1.3.3 Association Rules with both chosen Confidence and Lift values 

### 1.4. Take a Look at Maximal Patterns: Compute Maximal Frequent Itemsets
- discuss their utility compared to frequent patterns
- analyse the association rules they can unravel

In [47]:
# Write code in cells like this
# ....

Write text in cells like this ...


### 1.5 Conclusions from Mining Frequent Patterns in All Stores (Global Patterns and Rules)

Write text in cells like this ...


## 2. Mining Frequent Itemsets and Association Rules: Looking for Differences between Stores

The 24 stores, whose transactions were analysed in Task 1, are in fact from purchases carried out in **different types of stores**:
* Deluxe Supermarkets: STORE_ID = 8, 12, 13, 17, 19, 21
* Gourmet Supermarkets: STORE_ID = 4, 6
* Mid-Size Grocerys: STORE_ID = 9, 18, 20, 23
* Small Grocerys: STORE_ID = 2, 5, 14, 22
* Supermarkets: STORE_ID = 1, 3, 7, 10, 11, 15, 16

In this context, in this second task you should compute frequent itemsets and association rules for specific groups of stores (specific/local patterns), and then compare the store specific results with those obtained when all transactions were analysed independently of the type of store (global patterns). 

**The goal is to find similarities and differences in buying patterns according to the types of store. Do popular products change? Are there buying patterns specific to the type of store?**

### 2.1. Analyse Deluxe Supermarkets and Gourmet Supermarkets

Here you should analyse **both** the transactions from **Deluxe Supermarkets (STORE_ID = 8, 12, 13, 17, 19, 21)** and **Gourmet Supermarkets (STORE_ID = 4, 6)**.

#### 2.1.1. Load/Preprocess the Dataset

**You might need to change a bit the preprocessing, although most of it should be reused.**

In [268]:
# Write code in cells like this
# ....

Write text in cells like this ...


#### 2.1.2. Compute Frequent Itemsets

**This should be trivial now.**

In [273]:
# Write code in cells like this
# ....

Write text in cells like this ...


#### 2.1.3. Generate Association Rules from Frequent Itemsets

**This should be trivial now.**

In [277]:
# Write code in cells like this
# ....

Write text in cells like this 

#### 2.1.4.  Take a look at Maximal Patterns

In [281]:
# Write code in cells like this
# ....

Write text in cells like this 

#### 2.1.5.  Deluxe/Gourmet Supermarkets versus All Stores (Global versus Deluxe/Gourmet Supermarkets Specific Patterns and Rules)

Discuss the similarities and diferences between the results obtained in task 1. (frequent itemsets and association rules found in transactions from all stores) and those obtained above (frequent itemsets and association rules found in transactions only from Deluxe/Gourmet Supermarkets).


In [164]:
# Write code in cells like this
# ....

Write text in cells like this 

### 2.2. Analyse Small Groceries

Here you should analyse **Small Groceries (STORE_ID = 2, 5, 14, 22)**.

#### 2.2.1.  Load/Preprocess the Dataset

**This should be trivial now.**

In [174]:
# Write code in cells like this
# ....

Write text in cells like this 


#### 2.2.2. Compute Frequent Itemsets

Write text in cells like this 


In [168]:
# Write code in cells like this
# ....

#### 2.2.3. Generate Association Rules from Frequent Itemsets

In [168]:
# Write code in cells like this
# ....

Write text in cells like this


#### 2.2.4. Take a Look at Maximal Patterns

In [172]:
# Write code in cells like this
# ....

Write text in cells like this


#### 2.2.5. Small Groceries versus All Stores (Global versus Small Groceries Specific Patterns and Rules)

Discuss the similarities and diferences between the results obtained in task 1. (frequent itemsets and association rules found in transactions from all stores) and those obtained above (frequent itemsets and association rules found in transactions only Small Groceries).

Write text in cells like this


### 2.3.  Deluxe/Gourmet Supermarkets versus Small Groceries

Discuss the similarities and diferences between the results obtained in task 2.1. (frequent itemsets and association rules found in transactions only from Deluxe/Gourmet Supermarkets) and those obtained in task 2.2. (frequent itemsets and association rules found in transactions only Small Groceries).

Write text in cells like this